In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf,split,explode
from pyspark.ml.feature import NGram
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType

In [2]:
spark = SparkSession.\
        builder.\
        appName("comment-anlysis").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

23/01/27 17:39:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [36]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf,explode
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType
import re

data = spark.read.parquet('hdfs://namenode:9000/tiki/Product')

schema = StructType([ 
    StructField("id",IntegerType(),True), 
    StructField("master_id",IntegerType(),True), 
    StructField("sku",StringType(),True), 
    StructField("name",StringType(),True),
    StructField("short_description",StringType(),True), 
    StructField("price",IntegerType(),True),
    StructField("list_price",IntegerType(),True),
    StructField('original_price', IntegerType(),True),
    StructField('discount', IntegerType(),True),
    StructField('discount_rate', FloatType(),True),
    
    StructField("rating_average",FloatType(),True), 
    StructField("review_count",IntegerType(),True), 
    StructField("productset_group_name",StringType(),True), 
    StructField("all_time_quantity_sold",IntegerType(),True),
    
    StructField("description",StringType(),True), 
    StructField("current_seller",MapType(StringType(),StringType()),True),
    StructField("other_sellers",ArrayType(MapType(StringType(),StringType())),True),
    StructField("breadcrumbs",ArrayType(MapType(StringType(),StringType())),True),
    StructField("specifications",ArrayType(StructType(
                    [
                        StructField("name", StringType()),
                        StructField("attributes",ArrayType(MapType(StringType(),StringType())),True),
                    ]
                )),True),

    StructField('return_and_exchange_policy', StringType(),True)
])

df = data.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*")

df.createOrReplaceTempView('Product')
def parserAtt(specifications):
    result = ""
    try:
        for s in specifications:
            for a in s.attributes:
                result += a['value']
        result = cleanText(result)
    except:
        return ""
    return result

spark.udf.register('parserAtt',parserAtt,StringType())



CLEANR = re.compile('<.*?>') 
def cleanText(str_raw):
    # remove tags html
    str_raw = re.sub(CLEANR, ' ', str_raw)

    # remove special character
    str_raw = re.sub('\W+', ' ', str_raw)
    
    # remove number
    str_raw = re.sub("[0-9]+", "", str_raw)
    
    # remove space
    cleantext = re.sub(" +", " ", str_raw)
    return cleantext.lower()
spark.udf.register("cleanText", cleanText,StringType())

product_clean = spark.sql("""
        select * from (
            select id,master_id,sku,price,list_price,original_price,discount,discount_rate,
            rating_average,review_count,productset_group_name,all_time_quantity_sold,
            name, short_description,
            cleanText(name) clean_name,cleanText(description) clean_description,parserAtt(specifications) clean_specifications,
            breadcrumbs[0].name category_name,breadcrumbs[0].category_id category_id,
            current_seller.id seller_id,current_seller.name seller_name,current_seller.store_id seller_store_id,
            cast(current_seller.product_id as int) spid,
            row_number() over (partition by id,current_seller.id order by all_time_quantity_sold desc) cnt
            from Product
        ) where cnt = 1
""")

23/01/27 17:32:46 WARN SimpleFunctionRegistry: The function parseratt replaced a previously registered function.


In [ ]:
df = spark.read.parquet('hdfs://namenode:9000/analysis/Product_Shop')

In [3]:
df = spark.read.parquet('hdfs://namenode:9000/analysis/Product_Shop')

In [37]:
product_clean.createOrReplaceTempView('test')

In [10]:
df.createOrReplaceTempView('product_shop')

In [7]:
df

DataFrame[p_id: int, master_id: int, price: int, list_price: int, original_price: int, discount: int, discount_rate: float, productset_group_name: string, all_time_quantity_sold: int, p_name: string, short_description: string, p_clean_name: string, clean_description: string, clean_specifications: string, category_name: string, seller_id: string, store_id: int, s_name: string, icon: string, url: string, is_official: boolean, store_level: string, is_followed: boolean, avg_rating_point: float, review_count: int, total_follower: int, days_since_joined: int, category_id: int]

In [ ]:
    select category_name,sum(all_time_quantity_sold*price/1000000000) price
    from product_shop
    group by category_name
    order by price desc

In [43]:
spark.sql("""
    select id,seller_id,all_time_quantity_sold,seller_name
    from test
    where name = 'Sách OSHO Yêu - Being In Love - Yêu Trong Tỉnh Thức'
""").show(30)

+--------+---------+----------------------+--------------------+
|      id|seller_id|all_time_quantity_sold|         seller_name|
+--------+---------+----------------------+--------------------+
|75307228|   146243|                  null|           ETS Books|
|75307228|     1308|                     8|Tazano Official S...|
|75307228|   161219|                    10| Nhà Sách Trẻ Online|
|75307228|    60209|                  null|              VBooks|
|75307228|   111899|                     5|       Sống Official|
|75307228|   126688|                     1|           info book|
|75307228|    97533|                  null|  Nhà Sách Vĩnh Thụy|
|75307228|     2918|                   158|          First News|
|75307228|   281062|                  null|        Bamboo Books|
|75307228|    26874|                   752|   Phương Đông Books|
|75307228|    53660|                  1308|     Nhà sách Fahasa|
|75307228|   159932|                  null|              Hodico|
|75307228|   145814|     

In [31]:
spark.sql("""
    select p_name,sum(all_time_quantity_sold) cnt
    from product_shop
    group by  p_name
    order by cnt desc
""").show(30)

+--------------------+-------+
|              p_name|    cnt|
+--------------------+-------+
|Sách OSHO Yêu - B...|1404791|
|Phiếu Quà Tặng Ti...|1154744|
|Phiếu Quà Tặng Ti...|1112757|
|Phiếu Quà Tặng Ti...| 747354|
|Tã Quần Caryn Loạ...| 643957|
|Phiếu Quà Tặng Ti...| 200277|
|Nóng Giận Là Bản ...| 199953|
|Dầu Đậu Nành Simp...| 124426|
|Dầu Gội Chống Gàu...| 118604|
|Cây Cam Ngọt Của Tôi| 116145|
|Yêu Những Điều Kh...| 105617|
|Phiếu Quà Tặng Ti...| 104986|
|Đàn Ông Sao Hỏa Đ...| 102671|
|         Dám Bị Ghét|  95314|
|Bước Chậm Lại Giữ...|  92826|
|Thùng 48 Hộp Sữa ...|  77529|
|Tập Học Sinh Tiki...|  72297|
|Thiên Tài Bên Trá...|  66087|
|Sách Thay Đổi Cuộ...|  65293|
|Sách Hiểu Về Trái...|  62345|
|Khi Hơi Thở Hóa T...|  60299|
|Phiếu Quà Tặng Ti...|  60137|
|Phiếu Quà Tặng Ti...|  57861|
|Nhà Giả Kim (Tái ...|  56401|
|Phiếu Quà Tặng Ti...|  55824|
|Rèn Luyện Tư Duy ...|  55676|
|Giấy Thấm Dầu Acn...|  54780|
|Dầu Ăn Thượng Hạn...|  54590|
|Máy Vắt Cam Lock&...|  54234|
|Điều Kỳ

In [29]:
spark.sql("""
    select category_name,
sum(price*10)
    from product_shop
group by category_name
order by sum(price*nvl(all_time_quantity_sold,0)) desc


""").show(30)

+--------------------+-----------------+
|       category_name|sum((price * 10))|
+--------------------+-----------------+
|       Nhà Sách Tiki|      17380842270|
|  Làm Đẹp - Sức Khỏe|      13738348600|
|       Điện Gia Dụng|      90235589970|
| Điện Tử - Điện Lạnh|     790386271190|
|Thiết Bị Số - Phụ...|      19204293530|
|     Bách Hóa Online|       4494572660|
|   Đồ Chơi - Mẹ & Bé|      11296523750|
|  Nhà Cửa - Đời Sống|      12875110360|
|Laptop - Máy Vi T...|      48007707760|
|Ô Tô - Xe Máy - X...|      38562682930|
| Thể Thao - Dã Ngoại|       9868625900|
|Máy Ảnh - Máy Qua...|      56233583520|
|Điện Thoại - Máy ...|      51316154680|
|      Thời trang nam|       4783352710|
|        Balo và Vali|      12143709450|
|   Voucher - Dịch vụ|      34573799280|
|      Giày - Dép nam|      15657009640|
|       Thời trang nữ|       4203660110|
|Đồng hồ và Trang sức|      12396174990|
|  Túi thời trang nam|      12122045210|
|       Giày - Dép nữ|       8463675700|
| Phụ kiện thời 

In [3]:
df_product = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Product')
df_comment = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Comment')
df_shop = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/ShopInfo')

In [46]:
df_comment.take(2)

[Row(id=14537315, content='Sản phẩm đẹp', clean_content='sản phẩm đẹp', thank_count=0, customer_id=16914562, rating=5, spid=56941524, seller_id='1', product_id=56941522, customer_full_name='Hailey Lee', purchased_at='1641748169', date_purchased_at='2022-01-09', review_created_date='2022-01-18 11:57:39', delivery_date='2022-01-11 16:54:39', review_after_delivery=7, sentiment='Positive', lable=2),
 Row(id=9580902, content='khá là ok', clean_content='khá là ok', thank_count=0, customer_id=21190123, rating=5, spid=87226761, seller_id='1', product_id=87226754, customer_full_name='Văn Cần Trường', purchased_at='1618068725', date_purchased_at='2021-04-10', review_created_date='2021-05-08 16:51:19', delivery_date='2021-04-23 11:03:53', review_after_delivery=15, sentiment='Positive', lable=2)]

In [4]:
df_product.count()

94844

In [5]:
df_comment.count()

3568061

In [6]:
df_shop.count()

8686

In [4]:
df_shop.printSchema()

root
 |-- id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_official: boolean (nullable = true)
 |-- store_level: string (nullable = true)
 |-- is_followed: boolean (nullable = true)
 |-- avg_rating_point: float (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- total_follower: integer (nullable = true)
 |-- days_since_joined: integer (nullable = true)



In [5]:
df_comment.printSchema()

root
 |-- id: integer (nullable = true)
 |-- content: string (nullable = true)
 |-- clean_content: string (nullable = true)
 |-- thank_count: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- spid: integer (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- customer_full_name: string (nullable = true)
 |-- purchased_at: string (nullable = true)
 |-- date_purchased_at: string (nullable = true)
 |-- review_created_date: string (nullable = true)
 |-- delivery_date: string (nullable = true)
 |-- review_after_delivery: integer (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- lable: integer (nullable = true)



In [8]:
df_product.printSchema()

root
 |-- id: integer (nullable = true)
 |-- master_id: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- list_price: integer (nullable = true)
 |-- original_price: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- rating_average: float (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- productset_group_name: string (nullable = true)
 |-- all_time_quantity_sold: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- clean_name: string (nullable = true)
 |-- clean_description: string (nullable = true)
 |-- clean_specifications: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_store_id: string (nullable = true)
 |-- spid: integer (nullable = true)
 |-- category_id: integer (null

In [7]:
Product_Shop = spark.read.parquet('hdfs://namenode:9000/analysis/Product_Shop')
metaData = spark.read.parquet('hdfs://namenode:9000/analysis/metaData')
comment_1gram = spark.read.parquet('hdfs://namenode:9000/analysis/comment_1gram')
comment_2gram = spark.read.parquet('hdfs://namenode:9000/analysis/comment_2gram')

In [12]:
Product_Shop.count()

23/01/25 13:31:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


95451

In [13]:
metaData.count()

3038114

In [14]:
comment_1gram.count()

81038

In [15]:
comment_2gram.count()

1863421

In [10]:
comment_1gram.printSchema()

root
 |-- sentiment: string (nullable = true)
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [11]:
comment_2gram.printSchema()

root
 |-- sentiment: string (nullable = true)
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [8]:
Product_Shop.printSchema()

root
 |-- p_id: integer (nullable = true)
 |-- master_id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- list_price: integer (nullable = true)
 |-- original_price: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- productset_group_name: string (nullable = true)
 |-- all_time_quantity_sold: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- p_clean_name: string (nullable = true)
 |-- clean_description: string (nullable = true)
 |-- clean_specifications: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- s_name: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_official: boolean (nullable = true)
 |-- store_level: string (nullable = true)
 |-- is_followed: boolean (nullable = tr

In [9]:
metaData.printSchema()

root
 |-- p_id: integer (nullable = true)
 |-- master_id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- list_price: integer (nullable = true)
 |-- original_price: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- productset_group_name: string (nullable = true)
 |-- all_time_quantity_sold: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- p_clean_name: string (nullable = true)
 |-- clean_description: string (nullable = true)
 |-- clean_specifications: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- s_name: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_official: boolean (nullable = true)
 |-- store_level: string (nullable = true)
 |-- is_followed: boolean (nullable = tr

In [3]:
df_product = spark.read.parquet('hdfs://namenode:9000/tiki/Product')
df_comment = spark.read.parquet('hdfs://namenode:9000/tiki/Comment')
df_shop = spark.read.parquet('hdfs://namenode:9000/tiki/ShopInfo')

In [22]:
df_product.show(10,120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   value|
+------------------------------------------------------------------------------------------------------------------------+
|{"id": 24629723, "master_id": 24629723, "sku": "3201131389423", "name": "K\u00ednh M\u00e1t Nam Cao C\u1ea5p K\u00e8m...|
|{"id": 614854, "master_id": 614854, "sku": "2510608412924", "name": "Nh\u1eefng Gi\u1ea5c M\u01a1 \u1ede Hi\u1ec7u S\...|
|{"id": 62231649, "master_id": 62231649, "sku": "6840385438104", "name": "Combo 3 c\u1eb7p thanh ch\u1eafn khe c\u1eed...|
|{"id": 151756158, "master_id": 151756158, "sku": "1823652985285", "name": "N\u1ed2I \u00c1P SU\u1ea4T BLUESTONE PCB-5...|
|{"id": 614854, "master_id": 614854, "sku": "2510608412924", "name": "Nh\u1eefng Gi\u1ea5c M\u01a1 \u1ede Hi\u1ec7u S\...|
|{"id": 24629723

In [23]:
df_comment.show(10,120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   value|
+------------------------------------------------------------------------------------------------------------------------+
|{"id": 12696578, "title": "B\u00ecnh th\u01b0\u1eddng", "content": "S\u1ea3n ph\u1ea9m Tiki Trading th\u00ec okie. M\...|
|{"id": 17837163, "title": "C\u1ef1c k\u00ec h\u00e0i l\u00f2ng", "content": "Pepsi chai nh\u1ecf. U\u1ed1ng ok nha. T...|
|{"id": 17656856, "title": "C\u1ef1c k\u00ec h\u00e0i l\u00f2ng", "content": "Giao h\u00e0ng nhanh, date m\u1edbi, th\...|
|{"id": 16628410, "title": "C\u1ef1c k\u00ec h\u00e0i l\u00f2ng", "content": "d\u00ed\u00edidididididiidid\u00ecvivivi...|
|{"id": 10268605, "title": "C\u1ef1c k\u00ec h\u00e0i l\u00f2ng", "content": "h\u00e0ng ngon ae nh\u00e9, n\u00ean mua...|
|{"id": 16265495

In [24]:
df_shop.show(10,120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   value|
+------------------------------------------------------------------------------------------------------------------------+
|{"id": 13239, "store_id": 12900, "name": "Jess shop cosmetics", "icon": "https://vcdn.tikicdn.com/ts/seller/d5/cd/ca/...|
|{"id": 232650, "store_id": 219682, "name": "Hasu Sago Shop", "icon": "https://vcdn.tikicdn.com/ts/seller/11/87/fe/d55...|
|{"id": 31606, "store_id": 30924, "name": "H89 store", "icon": "https://vcdn.tikicdn.com/ts/seller/86/23/f2/5a4de26395...|
|{"id": 251877, "store_id": 238909, "name": "Gi\u00e0y d\u00e9p RYNA", "icon": "https://vcdn.tikicdn.com/ts/seller/95/...|
|{"id": 230813, "store_id": 217845, "name": "ROC LTD", "icon": "https://vcdn.tikicdn.com/ts/seller/c8/34/61/27f356bfc2...|
|{"id": 276580, 

In [10]:
from pyspark.sql.functions import sum

In [16]:
df_product.select(sum('all_time_quantity_sold')).collect()

[Row(sum(all_time_quantity_sold)=25207914)]

In [17]:
df_product_shop.select(sum('all_time_quantity_sold')).collect()

[Row(sum(all_time_quantity_sold)=25126552)]

In [13]:
df_product.select('all_time_quantity_sold').take(10)

[Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=8),
 Row(all_time_quantity_sold=1),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None)]

In [12]:
df_product_shop.select('all_time_quantity_sold').take(10)

[Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=8),
 Row(all_time_quantity_sold=1),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None),
 Row(all_time_quantity_sold=None)]

In [ ]:
df_product_shop.all_time_quantity_sold.sum()